In [168]:
import pandas as pd
import json
import numpy
import geopy.distance
import plotly.express as px
import math
import sys

In [169]:
dataset = pd.read_csv("partial_added_points.csv")

In [170]:
len(dataset)

88816

In [171]:
# calculate the distance between the two points


def point_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    return dist

In [173]:
fig = px.line_mapbox(dataset[:800], lat="lat", lon="lon", color="id", zoom=3, height=700)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = 41.14,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [179]:
numpy.set_printoptions(threshold=sys.maxsize)
gap_penalty = -1


def create_matrix(track1, track2):
    matrix = numpy.zeros((len(track1) + 1, len(track2) + 1))

    for i in range(1, len(track1) + 1):
        t1 = track1[i - 1]
        j = 1
        while j < len(track2) + 1:
            t2 = track2[j - 1]
            dist = point_distance(t1, t2)
            if dist > 30:
                match = -1
            else:
                match = 1
            match = matrix[i - 1][j - 1] + match
            delete = matrix[i - 1][j] + gap_penalty
            insert = matrix[i][j - 1] + gap_penalty
            matrix[i, j] = max(0, max(match, max(delete, insert)))
            # If the number set in the field is zero, check the distance. If the distance is severe, skip a few points as you know they will not be matching either.
            if matrix[i, j] == 0.0:
                j = j + math.floor(dist / 40)
            j += 1
    return matrix


track1 = dataset.loc[dataset["id"] == "T1"].values.tolist()
track2 = dataset.loc[dataset["id"] == "T3"].values.tolist()
matrix = create_matrix(track1, track2)
print(matrix)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]
 [ 0.  1.  0.  

In [180]:
def backtrack(matrix, track1, track2):

    start = numpy.max(matrix)
    if start <= 3:
        return 0
    print(start)
    start_position = numpy.where(matrix == start)
    for max_value in range(len(start_position[0])):
        i = start_position[0][max_value]
        j = start_position[1][max_value]

        a1 = []

        while matrix[i][j] > 0:
            if matrix[i][j] == matrix[i - 1][j - 1] + 1:
                a1.insert(0, track1[i - 1])
                i = i - 1
                j = j - 1
                # print('match!', i, j)
            elif matrix[i][j] == matrix[i - 1][j - 1] - 1:
                i = i - 1
                j = j - 1
            elif matrix[i][j] == matrix[i - 1][j] - 1:
                a1.insert(0, track1[i - 1])
                i = i - 1
                # print('gap', i, j)
            elif matrix[i][j] == matrix[i][j - 1] - 1:
                j = j - 1
                # print('mismatch', i, j)
            else:
                a1 = []
                break
        print(a1)

In [181]:
track_ids = pd.read_csv("partial_ids.csv").values.tolist()
print(track_ids[:5])

[['T1'], ['T2'], ['T3'], ['T4'], ['T5']]


In [182]:
def Smith_Waterman(dataset):
    for i in range(len(track_ids)):
        iden = track_ids[i][0]
        track = dataset.loc[dataset["id"] == iden]
        print("track1: ", iden)
        for j in range(i + 1, len(track_ids)):
            print("track2: ", track_ids[j][0])
            next_track = dataset.loc[dataset["id"] == track_ids[j][0]]
            # print('track2: ', track_ids[j][0])
            matrix = create_matrix(track.values.tolist(), next_track.values.tolist())
            # print(matrix)
            backtrack(matrix, track.values.tolist(), next_track.values.tolist())

        # kjør alle funksjonene med kombinasjonen av denne og alle andre tracks.


Smith_Waterman(dataset)

track1:  T1
track2:  T2
track2:  T3
33.0
[[41.148855000000005, -8.585685000000002, 'T1'], [41.14892699999999, -8.585730000000002, 'T1'], [41.148963, -8.585982, 'T1'], [41.148963, -8.585982, 'T1'], [41.1489585, -8.586189000000001, 'T1'], [41.148954, -8.586396, 'T1'], [41.148954, -8.586396, 'T1'], [41.148837, -8.586234000000001, 'T1'], [41.14872, -8.586072000000001, 'T1'], [41.14872, -8.586072, 'T1'], [41.1485454, -8.5861224, 'T1'], [41.1483708, -8.586172800000002, 'T1'], [41.1481962, -8.586223200000003, 'T1'], [41.14802159999999, -8.586273600000004, 'T1'], [41.14784699999999, -8.586324000000005, 'T1'], [41.147847, -8.586324000000001, 'T1'], [41.14775025, -8.586492750000001, 'T1'], [41.1476535, -8.586661500000002, 'T1'], [41.14755675000001, -8.586830250000002, 'T1'], [41.14746000000001, -8.586999000000002, 'T1'], [41.14746, -8.586999, 'T1'], [41.147307, -8.5867875, 'T1'], [41.147154, -8.586575999999999, 'T1'], [41.14715400000001, -8.586575999999999, 'T1'], [41.14709500000001, -8.586388, 

29.0
[[41.148855000000005, -8.585685000000002, 'T1'], [41.14892699999999, -8.585730000000002, 'T1'], [41.148963, -8.585982, 'T1'], [41.148963, -8.585982, 'T1'], [41.1489585, -8.586189000000001, 'T1'], [41.148954, -8.586396, 'T1'], [41.148954, -8.586396, 'T1'], [41.148837, -8.586234000000001, 'T1'], [41.14872, -8.586072000000001, 'T1'], [41.14872, -8.586072, 'T1'], [41.1485454, -8.5861224, 'T1'], [41.1483708, -8.586172800000002, 'T1'], [41.1481962, -8.586223200000003, 'T1'], [41.14802159999999, -8.586273600000004, 'T1'], [41.14784699999999, -8.586324000000005, 'T1'], [41.147847, -8.586324000000001, 'T1'], [41.14775025, -8.586492750000001, 'T1'], [41.1476535, -8.586661500000002, 'T1'], [41.14755675000001, -8.586830250000002, 'T1'], [41.14746000000001, -8.586999000000002, 'T1'], [41.14746, -8.586999, 'T1'], [41.147307, -8.5867875, 'T1'], [41.147154, -8.586575999999999, 'T1'], [41.14715400000001, -8.586575999999999, 'T1'], [41.14709500000001, -8.586388, 'T1'], [41.14703600000001, -8.5862, 

track2:  T181
track2:  T182
track2:  T183
track2:  T184
track2:  T185
track2:  T186
track2:  T187
track2:  T188
track2:  T189
track2:  T190
track2:  T191
track2:  T192
track2:  T193
track2:  T194
track2:  T195
track2:  T196
track2:  T197
track2:  T198
track2:  T199
track2:  T200
track2:  T201
track2:  T202
track2:  T203
track2:  T204
track2:  T205
track2:  T206
track2:  T207
track2:  T208
track2:  T209
track2:  T210
track2:  T211
track2:  T212
track2:  T213
track2:  T214
track2:  T215
track2:  T216
track2:  T217
track2:  T218
track2:  T219
track2:  T220
track2:  T221
track2:  T222
track2:  T223
track2:  T224
track2:  T225
track2:  T226
track2:  T227
9.0
[[41.148522, -8.585676, 'T1'], [41.148639, -8.585712000000001, 'T1'], [41.148855000000005, -8.585685000000002, 'T1'], [41.148855000000005, -8.585685000000002, 'T1'], [41.14892699999999, -8.585730000000002, 'T1'], [41.148963, -8.585982, 'T1'], [41.148963, -8.585982, 'T1'], [41.1489585, -8.586189000000001, 'T1'], [41.148954, -8.586396, 'T

track2:  T44
track2:  T45
track2:  T46
track2:  T47
track2:  T48
track2:  T49
track2:  T50
track2:  T51
4.0
[[41.1508095, -8.606907, 'T2'], [41.15097899999999, -8.606817, 'T2'], [41.150979, -8.606817, 'T2'], [41.1511662, -8.6067252, 'T2']]
track2:  T52
track2:  T53
10.0
[[41.158585, -8.60452, 'T2'], [41.158779, -8.604519000000002, 'T2'], [41.158973, -8.604518000000002, 'T2'], [41.159167, -8.604517000000003, 'T2'], [41.159361, -8.604516000000004, 'T2'], [41.159555000000005, -8.604515000000005, 'T2'], [41.159749000000005, -8.604514000000005, 'T2'], [41.159943000000005, -8.604513000000006, 'T2'], [41.159943000000005, -8.604513, 'T2'], [41.16014700000001, -8.604468, 'T2']]
[[41.158585, -8.60452, 'T2'], [41.158779, -8.604519000000002, 'T2'], [41.158973, -8.604518000000002, 'T2'], [41.159167, -8.604517000000003, 'T2'], [41.159361, -8.604516000000004, 'T2'], [41.159555000000005, -8.604515000000005, 'T2'], [41.159749000000005, -8.604514000000005, 'T2'], [41.159943000000005, -8.604513000000006,

track2:  T132
track2:  T133
track2:  T134
track2:  T135
16.0
[[41.14605, -8.610596999999999, 'T2'], [41.146191, -8.610443999999998, 'T2'], [41.146191, -8.610444, 'T2'], [41.1462855, -8.610277499999999, 'T2'], [41.14638, -8.610111, 'T2'], [41.1464745, -8.609944500000001, 'T2'], [41.14656900000001, -8.609778000000002, 'T2'], [41.14666350000001, -8.609611500000003, 'T2'], [41.14675800000001, -8.609445000000004, 'T2'], [41.146758, -8.609445000000001, 'T2'], [41.146878, -8.609262000000001, 'T2'], [41.146998, -8.609079000000001, 'T2'], [41.147118000000006, -8.608896000000001, 'T2'], [41.147118, -8.608896, 'T2'], [41.147127, -8.608968, 'T2'], [41.1473295, -8.6088375, 'T2']]
track2:  T136
39.0
[[41.1462855, -8.610277499999999, 'T2'], [41.14638, -8.610111, 'T2'], [41.1464745, -8.609944500000001, 'T2'], [41.14656900000001, -8.609778000000002, 'T2'], [41.14666350000001, -8.609611500000003, 'T2'], [41.14675800000001, -8.609445000000004, 'T2'], [41.146758, -8.609445000000001, 'T2'], [41.146878, -8.

...

track2:  T153
track2:  T154
track2:  T155
track2:  T156
track2:  T157
track2:  T158
track2:  T159
track2:  T160
track2:  T161
track2:  T162
track2:  T163
track2:  T164
track2:  T166
track2:  T167
8.0
[[41.148558, -8.585739, 'T3'], [41.148828, -8.585730000000002, 'T3'], [41.148828, -8.585730000000002, 'T3'], [41.14897200000001, -8.585721, 'T3'], [41.148987000000005, -8.58591, 'T3'], [41.149002, -8.586099, 'T3'], [41.149017, -8.586288000000001, 'T3'], [41.14901700000001, -8.586288, 'T3']]
track2:  T168
track2:  T169
track2:  T170
track2:  T171
track2:  T172
track2:  T173
track2:  T174
144.0
[[41.14732499999998, -8.587151999999998, 'T3'], [41.147325, -8.587152, 'T3'], [41.14726424999999, -8.586968625, 'T3'], [41.14720349999999, -8.586785250000002, 'T3'], [41.14714274999999, -8.586601875000003, 'T3'], [41.14708199999998, -8.586418500000004, 'T3'], [41.14702124999998, -8.586235125000005, 'T3'], [41.14696049999998, -8.586051750000006, 'T3'], [41.14689974999997, -8.585868375000008, 'T3'], [41

track2:  T225
track2:  T226
track2:  T227
8.0
[[41.148558, -8.585739, 'T3'], [41.148828, -8.585730000000002, 'T3'], [41.148828, -8.585730000000002, 'T3'], [41.14897200000001, -8.585721, 'T3'], [41.148987000000005, -8.58591, 'T3'], [41.149002, -8.586099, 'T3'], [41.149017, -8.586288000000001, 'T3'], [41.14901700000001, -8.586288, 'T3']]
track2:  T228
50.0
[[41.14732499999998, -8.587151999999998, 'T3'], [41.147325, -8.587152, 'T3'], [41.14726424999999, -8.586968625, 'T3'], [41.14720349999999, -8.586785250000002, 'T3'], [41.14714274999999, -8.586601875000003, 'T3'], [41.14708199999998, -8.586418500000004, 'T3'], [41.14702124999998, -8.586235125000005, 'T3'], [41.14696049999998, -8.586051750000006, 'T3'], [41.14689974999997, -8.585868375000008, 'T3'], [41.14683899999997, -8.585685000000009, 'T3'], [41.146839, -8.585685000000002, 'T3'], [41.146759125, -8.5855095, 'T3'], [41.146679250000005, -8.585334, 'T3'], [41.14659937500001, -8.585158499999999, 'T3'], [41.14651950000001, -8.5849829999999

track2:  T302
track2:  T303
track2:  T304
track2:  T305
track2:  T306
track2:  T307
track2:  T308
63.0
[[41.147443799999984, -8.586977399999999, 'T3'], [41.14732499999998, -8.587151999999998, 'T3'], [41.147325, -8.587152, 'T3'], [41.14726424999999, -8.586968625, 'T3'], [41.14720349999999, -8.586785250000002, 'T3'], [41.14714274999999, -8.586601875000003, 'T3'], [41.14708199999998, -8.586418500000004, 'T3'], [41.14702124999998, -8.586235125000005, 'T3'], [41.14696049999998, -8.586051750000006, 'T3'], [41.14689974999997, -8.585868375000008, 'T3'], [41.14683899999997, -8.585685000000009, 'T3'], [41.146839, -8.585685000000002, 'T3'], [41.146759125, -8.5855095, 'T3'], [41.146679250000005, -8.585334, 'T3'], [41.14659937500001, -8.585158499999999, 'T3'], [41.14651950000001, -8.584982999999998, 'T3'], [41.14643962500001, -8.584807499999997, 'T3'], [41.146359750000016, -8.584631999999996, 'T3'], [41.14627987500002, -8.584456499999995, 'T3'], [41.14620000000002, -8.584280999999994, 'T3'], [41.14

12.0
[[41.141169, -8.613963, 'T4'], [41.141124000000005, -8.614125000000001, 'T4'], [41.1410844, -8.614317600000001, 'T4'], [41.1410448, -8.614510200000002, 'T4'], [41.1410052, -8.614702800000002, 'T4'], [41.1409656, -8.614895400000002, 'T4'], [41.140926, -8.615088000000002, 'T4'], [41.140926, -8.615088, 'T4'], [41.140818, -8.615276999999999, 'T4'], [41.140818, -8.615276999999999, 'T4'], [41.140809, -8.615259, 'T4'], [41.1408, -8.615241000000001, 'T4']]
track2:  T41
track2:  T42
track2:  T43
track2:  T44
track2:  T45
track2:  T46
track2:  T47
track2:  T48
track2:  T49
track2:  T50
track2:  T51
track2:  T52
track2:  T53
track2:  T54
track2:  T55
track2:  T56
track2:  T57
track2:  T58
track2:  T59
track2:  T60
track2:  T61
track2:  T62
track2:  T63
track2:  T64
track2:  T65
track2:  T66
track2:  T67
track2:  T68
track2:  T69
track2:  T70
track2:  T71
track2:  T72
track2:  T73
track2:  T74
track2:  T75
track2:  T76
track2:  T77
track2:  T78
track2:  T79
track2:  T80
track2:  T81
track2:  

KeyboardInterrupt: 

In [38]:
track1 = dataset.loc[dataset['id'] == 'T1']
track2 = dataset.loc[dataset['id'] == 'T3']
len(track1)

# matrix = create_matrix(track1.values.tolist(), track2.values.tolist())
# backtrack(matrix, track1.values.tolist(), track2.values.tolist())

36

In [163]:
list1 = [[41.148963, -8.585982, 'T1'], [41.1489585, -8.586189000000001, 'T1'], [41.148954, -8.586396, 'T1'], [41.148954, -8.586396, 'T1'], [41.148837, -8.586234000000001, 'T1'], [41.14872, -8.586072000000001, 'T1'], [41.14872, -8.586072, 'T1'], [41.1485454, -8.5861224, 'T1'], [41.1483708, -8.586172800000002, 'T1'], [41.1481962, -8.586223200000003, 'T1'], [41.14802159999999, -8.586273600000004, 'T1'], [41.14784699999999, -8.586324000000005, 'T1'], [41.147847, -8.586324000000001, 'T1'], [41.14775025, -8.586492750000001, 'T1'], [41.1476535, -8.586661500000002, 'T1'], [41.14755675000001, -8.586830250000002, 'T1'], [41.14746000000001, -8.586999000000002, 'T1'], [41.14746, -8.586999, 'T1'], [41.147307, -8.5867875, 'T1'], [41.147154, -8.586575999999999, 'T1'], [41.14715400000001, -8.586575999999999, 'T1'], [41.14709500000001, -8.586388, 'T1'], [41.14703600000001, -8.5862, 'T1'], [41.14697700000001, -8.586012, 'T1'], [41.14691800000001, -8.585824, 'T1'], [41.146859000000006, -8.585636000000001, 'T1'], [41.146800000000006, -8.585448000000001, 'T1'], [41.146741000000006, -8.585260000000002, 'T1'], [41.146682000000006, -8.585072000000002, 'T1'], [41.146623000000005, -8.584884000000002, 'T1'], [41.1489, -8.585640000000001, 'T7'], [41.148945, -8.585919, 'T7'], [41.148945, -8.585919, 'T7'], [41.14893599999999, -8.586144, 'T7'], [41.14892699999999, -8.586369000000001, 'T7'], [41.14892699999999, -8.586369000000001, 'T7'], [41.1487425, -8.5862475, 'T7'], [41.148558, -8.586126, 'T7'], [41.148558, -8.586126, 'T7'], [41.148348, -8.58618, 'T7'], [41.148138, -8.586234000000001, 'T7'], [41.14792799999999, -8.586288000000001, 'T7'], [41.147928, -8.586288, 'T7'], [41.147823, -8.586441, 'T7'], [41.147718000000005, -8.586594000000002, 'T7'], [41.14761300000001, -8.586747000000003, 'T7'], [41.147613, -8.586747, 'T7'], [41.147415, -8.586769499999999, 'T7'], [41.147217000000005, -8.586791999999999, 'T7'], [41.147217000000005, -8.586791999999999, 'T7'], [41.147170200000005, -8.586611999999999, 'T7'], [41.147123400000005, -8.586431999999999, 'T7'], [41.147076600000005, -8.586251999999998, 'T7'], [41.147029800000006, -8.586071999999998, 'T7'], [41.146983000000006, -8.585891999999998, 'T7'], [41.146936200000006, -8.585711999999997, 'T7'], [41.146889400000006, -8.585531999999997, 'T7'], [41.14684260000001, -8.585351999999997, 'T7'], [41.14679580000001, -8.585171999999996, 'T7'], [41.14674900000001, -8.584991999999996, 'T7']]
list2 = [[41.147307, -8.5867875, 'T1'], [41.147154, -8.586575999999999, 'T1'], [41.14715400000001, -8.586575999999999, 'T1'], [41.14709500000001, -8.586388, 'T1'], [41.14703600000001, -8.5862, 'T1'], [41.14697700000001, -8.586012, 'T1'], [41.14691800000001, -8.585824, 'T1'], [41.146859000000006, -8.585636000000001, 'T1'], [41.146800000000006, -8.585448000000001, 'T1'], [41.146741000000006, -8.585260000000002, 'T1'], [41.146682000000006, -8.585072000000002, 'T1'], [41.146623000000005, -8.584884000000002, 'T1'], [41.147217000000005, -8.586791999999999, 'T7'], [41.147170200000005, -8.586611999999999, 'T7'], [41.147123400000005, -8.586431999999999, 'T7'], [41.147076600000005, -8.586251999999998, 'T7'], [41.147029800000006, -8.586071999999998, 'T7'], [41.146983000000006, -8.585891999999998, 'T7'], [41.146936200000006, -8.585711999999997, 'T7'], [41.146889400000006, -8.585531999999997, 'T7'], [41.14684260000001, -8.585351999999997, 'T7'], [41.14679580000001, -8.585171999999996, 'T7'], [41.14674900000001, -8.584991999999996, 'T7'], [41.146749, -8.584992, 'T7']]

matching = [[41.148855000000005, -8.585685000000002, 'T1'], [41.14892699999999, -8.585730000000002, 'T1'], [41.148963, -8.585982, 'T1'], [41.148963, -8.585982, 'T1'], [41.1489585, -8.586189000000001, 'T1'], [41.148954, -8.586396, 'T1'], [41.148954, -8.586396, 'T1'], [41.148837, -8.586234000000001, 'T1'], [41.14872, -8.586072000000001, 'T1'], [41.14872, -8.586072, 'T1'], [41.1485454, -8.5861224, 'T1'], [41.1483708, -8.586172800000002, 'T1'], [41.1481962, -8.586223200000003, 'T1'], [41.14802159999999, -8.586273600000004, 'T1'], [41.14784699999999, -8.586324000000005, 'T1'], [41.147847, -8.586324000000001, 'T1'], [41.14775025, -8.586492750000001, 'T1'], [41.1476535, -8.586661500000002, 'T1'], [41.14755675000001, -8.586830250000002, 'T1'], [41.14746000000001, -8.586999000000002, 'T1'], [41.14746, -8.586999, 'T1'], [41.147307, -8.5867875, 'T1'], [41.147154, -8.586575999999999, 'T1'], [41.14715400000001, -8.586575999999999, 'T1'], [41.14709500000001, -8.586388, 'T1'], [41.14703600000001, -8.5862, 'T1'], [41.14697700000001, -8.586012, 'T1'], [41.14691800000001, -8.585824, 'T1'], [41.146859000000006, -8.585636000000001, 'T1'], [41.146800000000006, -8.585448000000001, 'T1'], [41.146741000000006, -8.585260000000002, 'T1'], [41.146682000000006, -8.585072000000002, 'T1'], [41.146623000000005, -8.584884000000002, 'T1'], [41.148828, -8.585730000000002, 'T3'], [41.148828, -8.585730000000002, 'T3'], [41.14897200000001, -8.585721, 'T3'], [41.148987000000005, -8.58591, 'T3'], [41.149002, -8.586099, 'T3'], [41.149017, -8.586288000000001, 'T3'], [41.14901700000001, -8.586288, 'T3'], [41.148792, -8.586202499999999, 'T3'], [41.14856699999999, -8.586116999999998, 'T3'], [41.14856699999999, -8.586117, 'T3'], [41.148315, -8.586198, 'T3'], [41.148315, -8.586198, 'T3'], [41.148117, -8.5862385, 'T3'], [41.147919, -8.586279000000001, 'T3'], [41.147919, -8.586279000000001, 'T3'], [41.14780019999999, -8.5864536, 'T3'], [41.14768139999999, -8.5866282, 'T3'], [41.14756259999999, -8.5868028, 'T3'], [41.147443799999984, -8.586977399999999, 'T3'], [41.14732499999998, -8.587151999999998, 'T3'], [41.147325, -8.587152, 'T3'], [41.14726424999999, -8.586968625, 'T3'], [41.14720349999999, -8.586785250000002, 'T3'], [41.14714274999999, -8.586601875000003, 'T3'], [41.14708199999998, -8.586418500000004, 'T3'], [41.14702124999998, -8.586235125000005, 'T3'], [41.14696049999998, -8.586051750000006, 'T3'], [41.14689974999997, -8.585868375000008, 'T3'], [41.14683899999997, -8.585685000000009, 'T3'], [41.146839, -8.585685000000002, 'T3'], [41.146759125, -8.5855095, 'T3'], [41.146679250000005, -8.585334, 'T3'], [41.14659937500001, -8.585158499999999, 'T3']]
matching = [[41.148855000000005, -8.585685000000002, 'T1'], [41.14892699999999, -8.585730000000002, 'T1'], [41.148963, -8.585982, 'T1'], [41.148963, -8.585982, 'T1'], [41.1489585, -8.586189000000001, 'T1'], [41.148954, -8.586396, 'T1'], [41.148954, -8.586396, 'T1'], [41.148837, -8.586234000000001, 'T1'], [41.14872, -8.586072000000001, 'T1'], [41.14872, -8.586072, 'T1'], [41.1485454, -8.5861224, 'T1'], [41.1483708, -8.586172800000002, 'T1'], [41.1481962, -8.586223200000003, 'T1'], [41.14802159999999, -8.586273600000004, 'T1'], [41.14784699999999, -8.586324000000005, 'T1'], [41.147847, -8.586324000000001, 'T1'], [41.14775025, -8.586492750000001, 'T1'], [41.1476535, -8.586661500000002, 'T1'], [41.14755675000001, -8.586830250000002, 'T1'], [41.14746000000001, -8.586999000000002, 'T1'], [41.14746, -8.586999, 'T1'], [41.147307, -8.5867875, 'T1'], [41.147154, -8.586575999999999, 'T1'], [41.14715400000001, -8.586575999999999, 'T1'], [41.14709500000001, -8.586388, 'T1'], [41.14703600000001, -8.5862, 'T1'], [41.14697700000001, -8.586012, 'T1'], [41.14691800000001, -8.585824, 'T1'], [41.146859000000006, -8.585636000000001, 'T1'], [41.146800000000006, -8.585448000000001, 'T1'], [41.146741000000006, -8.585260000000002, 'T1'], [41.146682000000006, -8.585072000000002, 'T1'], [41.146623000000005, -8.584884000000002, 'T1'], [41.148828, -8.585730000000002, 'T3'], [41.148828, -8.585730000000002, 'T3'], [41.14897200000001, -8.585721, 'T3'], [41.148987000000005, -8.58591, 'T3'], [41.149002, -8.586099, 'T3'], [41.149017, -8.586288000000001, 'T3'], [41.14901700000001, -8.586288, 'T3'], [41.148792, -8.586202499999999, 'T3'], [41.14856699999999, -8.586116999999998, 'T3'], [41.14856699999999, -8.586117, 'T3'], [41.148315, -8.586198, 'T3'], [41.148315, -8.586198, 'T3'], [41.148117, -8.5862385, 'T3'], [41.147919, -8.586279000000001, 'T3'], [41.147919, -8.586279000000001, 'T3'], [41.14780019999999, -8.5864536, 'T3'], [41.14768139999999, -8.5866282, 'T3'], [41.14756259999999, -8.5868028, 'T3'], [41.147443799999984, -8.586977399999999, 'T3'], [41.14732499999998, -8.587151999999998, 'T3'], [41.147325, -8.587152, 'T3'], [41.14726424999999, -8.586968625, 'T3'], [41.14720349999999, -8.586785250000002, 'T3'], [41.14714274999999, -8.586601875000003, 'T3'], [41.14708199999998, -8.586418500000004, 'T3'], [41.14702124999998, -8.586235125000005, 'T3'], [41.14696049999998, -8.586051750000006, 'T3'], [41.14689974999997, -8.585868375000008, 'T3'], [41.14683899999997, -8.585685000000009, 'T3'], [41.146839, -8.585685000000002, 'T3'], [41.146759125, -8.5855095, 'T3'], [41.146679250000005, -8.585334, 'T3'], [41.14659937500001, -8.585158499999999, 'T3']]
new_points = [[41.148522, -8.585676], [41.148638999999996, -8.585712000000001], [41.148638999999996, -8.585712000000001], [41.148747, -8.585698500000001], [41.148855000000005, -8.585685000000002], [41.148855000000005, -8.585685000000002], [41.14892699999999, -8.585730000000002], [41.148945, -8.585856], [41.148962999999995, -8.585981999999998], [41.148962999999995, -8.585982], [41.148959999999995, -8.58612], [41.148956999999996, -8.586257999999999], [41.148953999999996, -8.586395999999999], [41.148954, -8.586396], [41.148876, -8.586288], [41.148798, -8.586179999999999], [41.14872, -8.586071999999998], [41.14872, -8.586072], [41.148623, -8.5861], [41.148526000000004, -8.586128], [41.14842900000001, -8.586156], [41.14833200000001, -8.586184000000001], [41.148235000000014, -8.586212000000002], [41.14813800000002, -8.586240000000002], [41.14804100000002, -8.586268000000002], [41.147944000000024, -8.586296000000003], [41.14784700000003, -8.586324000000003], [41.147847, -8.586324000000001], [41.14779171428571, -8.58642042857143], [41.14773642857142, -8.586516857142858], [41.14768114285713, -8.586613285714286], [41.14762585714284, -8.586709714285714], [41.14757057142855, -8.586806142857142], [41.14751528571426, -8.58690257142857], [41.147459999999974, -8.586998999999999], [41.147459999999995, -8.586999], [41.1473835, -8.58689325], [41.147307, -8.5867875], [41.1472305, -8.58668175], [41.147154, -8.586576], [41.14715400000001, -8.586575999999999], [41.147118600000006, -8.586463199999999], [41.147083200000004, -8.586350399999999], [41.1470478, -8.586237599999999], [41.1470124, -8.586124799999999], [41.146977, -8.586011999999998], [41.1469416, -8.585899199999998], [41.1469062, -8.585786399999998], [41.146870799999995, -8.585673599999998], [41.14683539999999, -8.585560799999998], [41.14679999999999, -8.585447999999998], [41.14676459999999, -8.585335199999998], [41.14672919999999, -8.585222399999997], [41.14669379999999, -8.585109599999997], [41.146658399999986, -8.584996799999997], [41.146622999999984, -8.584883999999997]]
t1 = [[41.148522, -8.585676], [41.148638999999996, -8.585712000000001], [41.148855000000005, -8.585685000000002], [41.148855000000005, -8.585685000000002], [41.14892699999999, -8.585730000000002], [41.148962999999995, -8.585982], [41.148962999999995, -8.585982], [41.148954, -8.586396], [41.148954, -8.586396], [41.14872, -8.586072], [41.14872, -8.586072], [41.148501749999994, -8.586135], [41.14828349999999, -8.586198000000001], [41.14806524999999, -8.586261000000002], [41.147846999999985, -8.586324000000003], [41.147847, -8.586324000000001], [41.147718, -8.586549000000002], [41.147588999999996, -8.586774000000002], [41.147459999999995, -8.586999000000002], [41.147459999999995, -8.586999], [41.147307, -8.5867875], [41.147154, -8.586575999999999], [41.14715400000001, -8.586575999999999], [41.14707814285715, -8.586334285714285], [41.14700228571429, -8.586092571428571], [41.146926428571426, -8.585850857142857], [41.146850571428566, -8.585609142857143], [41.146774714285705, -8.585367428571429], [41.146698857142844, -8.585125714285715], [41.146622999999984, -8.584884]]
t3 = [[41.148558, -8.585739], [41.148828, -8.585730000000002], [41.148828, -8.585730000000002], [41.14897200000001, -8.585721], [41.14899450000001, -8.5860045], [41.14901700000001, -8.586288], [41.14901700000001, -8.586288], [41.148792, -8.586202499999999], [41.14856699999999, -8.586116999999998], [41.14856699999999, -8.586117], [41.148315000000004, -8.586198], [41.148315000000004, -8.586198], [41.148117, -8.5862385], [41.147918999999995, -8.586279000000001], [41.147918999999995, -8.586279000000001], [41.14777049999999, -8.58649725], [41.14762199999999, -8.5867155], [41.14747349999999, -8.58693375], [41.14732499999999, -8.587152], [41.147324999999995, -8.587152], [41.14724399999999, -8.5869075], [41.14716299999999, -8.586663000000001], [41.14708199999999, -8.586418500000002], [41.14700099999999, -8.586174000000003], [41.14691999999999, -8.585929500000004], [41.146838999999986, -8.585685000000005], [41.146839, -8.585685000000002], [41.1467325, -8.585451], [41.146626, -8.585217], [41.1465195, -8.584983], [41.146412999999995, -8.584748999999999], [41.146306499999994, -8.584514999999998], [41.14619999999999, -8.584280999999997], [41.14620000000001, -8.584281], [41.146044428571436, -8.584108714285716], [41.145888857142864, -8.58393642857143], [41.14573328571429, -8.583764142857145], [41.14557771428572, -8.58359185714286], [41.14542214285715, -8.583419571428575], [41.14526657142858, -8.58324728571429], [41.14511100000001, -8.583075000000004], [41.145111, -8.583075000000001], [41.145073714285715, -8.582830714285715], [41.14503642857143, -8.582586428571428], [41.144999142857145, -8.582342142857142], [41.14496185714286, -8.582097857142855], [41.144924571428575, -8.581853571428569], [41.14488728571429, -8.581609285714283], [41.144850000000005, -8.581364999999996], [41.144850000000005, -8.581365], [41.14495012500001, -8.58113325], [41.14505025000001, -8.580901500000001], [41.145150375000014, -8.580669750000002], [41.14525050000002, -8.580438000000003], [41.14535062500002, -8.580206250000003], [41.14545075000002, -8.579974500000004], [41.145550875000026, -8.579742750000005], [41.14565100000003, -8.579511000000005], [41.145651, -8.579511], [41.145858000000004, -8.579348999999999], [41.145858000000004, -8.579348999999999], [41.146083000000004, -8.579290499999999], [41.146308000000005, -8.579232], [41.146308, -8.579232], [41.146500374999995, -8.579290499999999], [41.14669274999999, -8.579348999999999], [41.14688512499999, -8.579407499999999], [41.14707749999999, -8.579465999999998], [41.147269874999985, -8.579524499999998], [41.14746224999998, -8.579582999999998], [41.14765462499998, -8.579641499999997], [41.14784699999998, -8.579699999999997], [41.147847, -8.5797], [41.148031153846155, -8.579625230769231], [41.14821530769231, -8.579550461538462], [41.14839946153847, -8.579475692307692], [41.148583615384624, -8.579400923076923], [41.14876776923078, -8.579326153846154], [41.148951923076936, -8.579251384615384], [41.14913607692309, -8.579176615384615], [41.14932023076925, -8.579101846153845], [41.149504384615405, -8.579027076923076], [41.14968853846156, -8.578952307692306], [41.14987269230772, -8.578877538461537], [41.15005684615387, -8.578802769230768], [41.15024100000003, -8.578727999999998], [41.150241, -8.578728], [41.15040790909091, -8.578845818181819], [41.150574818181816, -8.578963636363637], [41.150741727272724, -8.579081454545456], [41.15090863636363, -8.579199272727275], [41.15107554545454, -8.579317090909093], [41.15124245454545, -8.579434909090912], [41.151409363636354, -8.57955272727273], [41.15157627272726, -8.57967054545455], [41.15174318181817, -8.579788363636368], [41.15191009090908, -8.579906181818187], [41.152076999999984, -8.580024000000005], [41.152077, -8.580024], [41.152212, -8.58019], [41.152347, -8.580356], [41.152482, -8.580522], [41.152617, -8.580688], [41.152752, -8.580854], [41.152887, -8.58102], [41.153022, -8.581186], [41.153157, -8.581352], [41.153292, -8.581518], [41.153292, -8.581517999999999], [41.153472, -8.581686], [41.153652, -8.581854], [41.153832, -8.582022], [41.153832, -8.582022], [41.154018, -8.582151], [41.154204, -8.582279999999999], [41.15439, -8.582408999999998], [41.15439000000001, -8.582409], [41.15456100000001, -8.582499], [41.15473200000001, -8.582589], [41.15490300000001, -8.582679], [41.154903000000004, -8.582679], [41.155128000000005, -8.582751], [41.155353000000005, -8.582823], [41.155578000000006, -8.582894999999999], [41.155578, -8.582895], [41.1557598, -8.582913000000001], [41.1559416, -8.582931000000002], [41.1561234, -8.582949000000003], [41.1563052, -8.582967000000004], [41.156487, -8.582985000000004], [41.156487, -8.582985], [41.156739, -8.582967], [41.156991000000005, -8.582949], [41.156991, -8.582949], [41.15718675, -8.582897249999998], [41.157382500000004, -8.582845499999998], [41.15757825000001, -8.582793749999997], [41.15777400000001, -8.582741999999996], [41.157774, -8.582742], [41.15797800000001, -8.582664], [41.15818200000001, -8.582586], [41.158386000000014, -8.582507999999999], [41.15838600000001, -8.582508], [41.1585795, -8.582427000000001], [41.158773, -8.582346000000001], [41.158773, -8.582346000000001], [41.158947, -8.582274000000002], [41.159121, -8.582202000000002], [41.159295, -8.582130000000003], [41.15929500000001, -8.582130000000001], [41.159485125, -8.582051250000001], [41.15967525000001, -8.581972500000001], [41.15986537500001, -8.58189375], [41.16005550000001, -8.581815], [41.160245625000016, -8.58173625], [41.16043575000002, -8.5816575], [41.16062587500002, -8.58157875], [41.160816000000025, -8.5815], [41.160816000000004, -8.5815], [41.161005900000006, -8.5814478], [41.16119580000001, -8.581395599999999], [41.16138570000001, -8.581343399999998], [41.16157560000001, -8.581291199999997], [41.161765500000016, -8.581238999999997], [41.16195540000002, -8.581186799999996], [41.16214530000002, -8.581134599999995], [41.16233520000002, -8.581082399999994], [41.162525100000025, -8.581030199999994], [41.16271500000003, -8.580977999999993], [41.162715, -8.580978], [41.162893714285715, -8.580897], [41.16307242857143, -8.580816], [41.16325114285715, -8.580735], [41.163429857142866, -8.580654000000001], [41.16360857142858, -8.580573000000001], [41.1637872857143, -8.580492000000001], [41.163966000000016, -8.580411000000002], [41.163965999999995, -8.580411000000002], [41.164001999999996, -8.580141000000001], [41.164038000000005, -8.579871], [41.164038000000005, -8.579870999999999], [41.164159500000004, -8.579690999999999], [41.164281, -8.579510999999998], [41.1644025, -8.579330999999998], [41.164524, -8.579150999999998], [41.1646455, -8.578970999999997], [41.164767, -8.578790999999997], [41.16476699999999, -8.578791], [41.16488579999999, -8.578557], [41.165004599999996, -8.578323], [41.1651234, -8.578088999999999], [41.1652422, -8.577854999999998], [41.165361000000004, -8.577620999999997], [41.165361000000004, -8.577621], [41.165415, -8.577513], [41.165739, -8.577342], [41.165739, -8.577342], [41.1659145, -8.577243], [41.16609, -8.577143999999999], [41.166265499999994, -8.577044999999998], [41.16644099999999, -8.576945999999998], [41.166441, -8.576946], [41.166573, -8.576772], [41.166705, -8.576598], [41.166837, -8.576424000000001], [41.166969, -8.576250000000002], [41.167101, -8.576076000000002], [41.167233, -8.575902000000003], [41.167233, -8.575902000000001], [41.1673545, -8.575652250000001], [41.167476, -8.575402500000001], [41.1675975, -8.57515275], [41.167719, -8.574903]]
overlap = pd.DataFrame(matching, columns =['lat', 'lon', 'id'])


In [164]:
fig = px.line_mapbox(overlap, lat="lat", lon="lon", color="id", zoom=3, height=700)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = 41.14,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()